# Quick Tour

---

This is a quick tour of `pybuck`---a package to support dimensional analysis in Python.

In [1]:
from pybuck import *


We are studying the *Reynolds' pipe flow problem* [1], which has physical inputs:

| Input | Symbol | Units |
|--|--|--|
| Fluid density | $\rho$ | $\frac{M}{L^3}$ |
| Fluid bulk velocity | $U$ | $\frac{L}{T}$ |
| Pipe diameter | $D$ | $L$ |
| Fluid dynamic viscosity | $\mu$ | $\frac{M}{LT}$ |
| Roughness lengthscale | $\epsilon$ | $L$ |

We construct the *dimension matrix* [2] with compact syntax. Zeros are automatically filled in.

In [2]:
df_dim = col_matrix(
    rho = dict(M=1, L=-3),
    U   = dict(L=1, T=-1),
    D   = dict(L=1),
    mu  = dict(M=1, L=-1, T=-1),
    eps = dict(L=1)
)
df_dim


,rowname,rho,U,D,mu,eps
0,L,-3,1,1,-1,1
1,T,0,-1,0,-1,0
2,M,1,0,0,1,0


This is a standardized way to represent the physical units of the inputs. We can check the physical units of a power-product of the inputs above through an inner product. Note that $\frac{1}{2}\rho U^2$ is *dynamic pressure*, so it should have the units of pressure $\left[\frac{M}{LT^2}\right]$. Dimensionless constants are ignored when performing dimensional analysis.

In [3]:
df_weights = col_matrix(q = dict(rho=1, U=2))
df_res = inner(df_dim, df_weights)
df_res


,rowname,q
0,L,-1
1,T,-2
2,M,1


We can compute a basis for the nullspace of `df_dim`; this subspace is called the *pi subspace*.

In [4]:
df_pi = pi_basis(df_dim)
df_pi


,rowname,pi0,pi1
0,rho,-0.521959,0.115207
1,U,-0.521959,0.115207
2,D,-0.413385,-0.632884
3,mu,0.521959,-0.115207
4,eps,-0.108575,0.748091


This shows us that the pi subspace is two-dimensional. The weights are fairly non-standard; a more familiar set are the *Reynolds number* and the *relative roughness*.

In [5]:
df_standard = col_matrix(
    Re = dict(rho=1, U=1, D=1, mu=-1), # Reynolds number
    R  = dict(eps=1, D=-1)             # Relative roughness
)
df_standard


,rowname,Re,R
0,rho,1,0
1,U,1,0
2,eps,0,1
3,D,1,-1
4,mu,-1,0


We can check that these two bases are the same by computing the *subspace angles* between them:

In [6]:
angles(df_standard, df_pi)

array([2.47244751e-16, 7.02099014e-17])

All subspace angles are zero to working precision; these two bases express the same subspace.

Since these are the same subspace, we can re-express the computed basis `df_pi` in terms of the standard basis `df_standard`. This will allow us to physically interpret the computationally-derived weights.

In [7]:
df_pi_prime = express(df_pi, df_standard)
df_pi_prime


,rowname,pi0,pi1
0,Re,-0.521959,0.115207
1,R,-0.108575,0.748091


This shows us that `pi0` is composed of both `Re` and `R`, while `pi1` is almost entirely aligned with `R`.

We can non-dimensionalize target quantities by providing their physical dimensions. If we wanted to non-dimensionalize a flow rate $[Q] = \frac{M}{L^3 T}$, we could produce the *canonical non-dimensionalizing factor* [2] via

In [8]:
df_flowrate = col_matrix(Q = dict(M=1, L=-3, T=-1))
df_nondim = nondim(df_flowrate, df_dim)
print(inner(df_dim, df_nondim))
df_nondim


  rowname    Q
0       L -3.0
1       T -1.0
2       M  1.0


,rowname,Q
0,rho,0.571429
1,U,0.571429
2,D,-0.714286
3,mu,0.428571
4,eps,-0.714286


The canonical factor has the desired physical units and is orthogonal to the pi subspace. We can verify orthogonality computing the subspace angles.

In [9]:
angles(df_nondim, df_pi)

array([1.57079633])

This is $90^{\circ}$; orthogonal to the pi subspace.

# Bibliography

---

[1] O. Reynolds, "An experimental investigation of the circumstances which determine whether the motion of water shall be direct or sinuous, and of the law of resistance in parallel channels" (1883) *Royal Society*

[2] Z. del Rosario, M. Lee, and G. Iaccarino, "Lurking Variable Detection via Dimensional Analysis" (2019) *SIAM/ASA Journal on Uncertainty Quantification*